In [3]:
# encoding: utf-8
import pandas as pd
from pandas import *
import numpy as np
from pymongo import MongoClient
import tushare as ts
import matplotlib.pyplot as plt


pd.set_option('display.width', None)  # 设置字符显示宽度
pd.set_option('display.max_rows', None)  # 设置显示最大行
pd.set_option('display.max_columns', None)  # 设置显示最大行

pro = ts.pro_api('c0cad8f56caba4e70702d606290d04f88514a6bef046f60d13144151')
df = pro.fut_holding( symbol='JD2008', exchange='DCE')
df=df.sort_values(['trade_date'],ascending=True)
df2=df.fillna(0)
jd=df2.loc[df2['broker']!='期货公司会员']
jd=jd.copy()
broker2=['宏源期货','方正中期','英大期货','格林大华','美尔雅期货']
jd['净持仓']=jd.apply(lambda x: x['long_hld'] - x['short_hld'], axis=1)
jd=jd[jd['trade_date']>='20200515']
jd.tail()

Exception: 抱歉，您没有访问该接口的权限，权限的具体详情访问：https://tushare.pro/document/1?doc_id=108。

In [ ]:
# sums =jd[['trade_date', 'symbol','broker','净持仓']]
net_position =jd.groupby(['trade_date', 'symbol'])['净持仓'].sum().reset_index(name='净空汇总')
sums = jd.copy()
# sums = sums[sums['symbol']=='JD2005']
times = sums[sums['trade_date']==sums['trade_date'].iloc[-1]]
# print(times)
brokers=times['broker']
df=pd.DataFrame()
for i in brokers:
    broker=[sums[sums['broker']==i]]
    df = df.append(broker)
    
df['净持仓变动量'] = df['净持仓'] - df['净持仓'].shift(1).fillna(0)
# print(df)
df= df[df['trade_date']==df['trade_date'].iloc[-1]]
sums = df.sort_values('净持仓')
sums = sums[sums['净持仓']!=0]
sums =sums[['trade_date', 'symbol','broker','净持仓','净持仓变动量']]
sums

In [ ]:
jd2=jd[jd['净持仓']<0]
sums =jd2.groupby(['trade_date', 'symbol'])['净持仓'].sum().reset_index(name='净空汇总')

df=pd.DataFrame()
for i in broker2:
    try:
        brokers = jd[jd['broker'] == i]
        df2=pd.DataFrame(brokers)
        df = df.append(df2)
    except:
        pass
sums2 = df.groupby(['trade_date', 'symbol'])['净持仓'].sum().reset_index(name='五少净空')
merge = pd.merge(sums2, sums, on=['trade_date', 'symbol'], how='outer').fillna(0)
merge['五少占比']=merge.apply(lambda x: x['五少净空']/x['净空汇总'], axis=1)
# merge=merge[merge['symbol']=='JD2005']
merge

In [ ]:
%matplotlib notebook
#二行即可搞定画图中文乱码
plt.rcParams['font.sans-serif']=['SimHei'] #用来正常显示中文标签
plt.rcParams['axes.unicode_minus']=False #用来正常显示负号
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
# 画图
plt.rcParams['font.sans-serif']=['SimHei'] #用来正常显示中文标签
plt.rcParams['axes.unicode_minus']=False #用来正常显示负号

a= pd.DataFrame({'五少占比':np.array(merge['五少占比']),'五少净空':np.array(merge['五少净空'])},index=merge['trade_date'])
ax = a.plot(
    secondary_y=['五少净空'],
    x_compat=True,
    grid=True)

ax.set_title(merge['symbol'].iloc[0]+" 五少 "+str(int(merge['五少净空'].iloc[-1]))+"手 日期:"+ merge['trade_date'].iloc[-1] )
ax.set_ylabel('占比')
ax.grid(linestyle="--", alpha=0.3)

ax.right_ax.set_ylabel('净空')
plt.show()